In [8]:
import re, json, time
from selenium import webdriver

# import By
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service

# from webdriver_manager.chrome import ChromeDriverManager
from parsel import Selector

import pandas as pd
import numpy as np


def scroll_page(url):
    # service = Service(ChromeDriverManager().install())

    # options = webdriver.ChromeOptions()
    # options.add_argument("--headless")
    # options.add_argument("--lang=en")
    # options.add_argument(
    #     "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36"
    # )

    driver = webdriver.Chrome()
    driver.get(url)

    # old_height = driver.execute_script(
    #     """
    #     function getHeight() {
    #         return document.querySelector('ytd-app').scrollHeight;
    #     }
    #     return getHeight();
    # """
    # )

    # while True:
    #     driver.execute_script(
    #         "window.scrollTo(0, document.querySelector('ytd-app').scrollHeight)"
    #     )

    #     time.sleep(2)

    #     new_height = driver.execute_script(
    #         """
    #         function getHeight() {
    #             return document.querySelector('ytd-app').scrollHeight;
    #         }
    #         return getHeight();
    #     """
    #     )

    #     if new_height == old_height:
    #         break

    #     old_height = new_height

    # time.sleep(3)
    selector = Selector(driver.page_source)
    driver.quit()

    return selector



In [9]:
# read channel IDs

with open("canais.txt", "r") as f:
    canais = f.readlines()


canais = [c.strip() for c in canais]
# take only the string after the last slash
canais = [c.split("/")[-1] for c in canais]

print(canais)


['@GUESS', '@calvinklein', '@usereserva', '@AnimaleBrasil']


In [10]:
base_url = "https://socialblade.com/youtube/channel/"

social_blade_data = []
for c in canais:
    response = scroll_page(base_url + c)

    # last 30 days views #socialblade-user-content > div:nth-child(23) > div:nth-child(3) > span
    views = response.css('#socialblade-user-content > div:nth-child(23) > div:nth-child(3) > span::text').get()
    likes = response.css("#socialblade-user-content > div:nth-child(23) > div:nth-child(2) > span::text").get()

    if views is None or likes is None:
        views = response.css('#socialblade-user-content > div:nth-child(16) > div:nth-child(3) > span::text').get()
        likes = response.css("#socialblade-user-content > div:nth-child(16) > div:nth-child(2) > span::text").get()



    social_blade_data.append({"Channel": c, "views": views, "likes": likes})


In [11]:
last_30days = pd.DataFrame(social_blade_data)

#socialblade-user-content > div:nth-child(16) > div:nth-child(3) > span

#socialblade-user-content > div:nth-child(16) > div:nth-child(2) > span

In [12]:
last_30days

,Channel,views,likes
0,@GUESS,"+265,586",+300
1,@calvinklein,"+2,272,474",+6K
2,@usereserva,"+54,371",+100
3,@AnimaleBrasil,"+3,351",+500


In [13]:
def fix_views(view: pd.Series) -> pd.Series:

    view = view.str.replace("+", "").str.replace(",", "")

    view = view.apply(lambda x: float(x.replace("K", "")) * 1000 if 'K' in x else x)
    view = view.apply(lambda x: float(x.replace("M", "")) * 1000000 if 'M' in str(x) else x)

    return view

last_30days = last_30days.assign(
    views=lambda x: fix_views(x["views"]),
    likes= lambda x: fix_views(x["likes"])


)

In [14]:
# create folder for lead

import os

os.makedirs(f"data/{canais[0].split('@')[-1]}", exist_ok=True)


In [15]:


last_30days.to_csv(f'data/{canais[0].split("@")[-1]}/last_30days.csv', index=False)

In [16]:
last_30days

,Channel,views,likes
0,@GUESS,265586,300
1,@calvinklein,2272474,6000.0
2,@usereserva,54371,100
3,@AnimaleBrasil,3351,500
